In [1]:
import os
import numpy as np
import cv2 as cv
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.applications.vgg16 import VGG16
import seaborn as sns
# import xgboost as xgb
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [2]:
def input_data(folder_path, output_data):    #importing the data into the output_data list
    for dirs in os.listdir(folder_path):
        class_name = dirs
        print(class_name)
        new_path = os.path.join(folder_path, class_name)
        for img in os.listdir(new_path):
            img_arr = cv.imread(os.path.join(new_path, img), cv.IMREAD_GRAYSCALE)
            resize = cv.resize(img_arr, (150,150))
            output_data.append([resize, class_name])
    return output_data

In [3]:
train_data = input_data(r'D:\datasets\AutismDataset\train', [])
test_data = input_data(r'D:\datasets\AutismDataset\test', [])

aut
Non
aut
Non


In [4]:
np.random.shuffle(train_data)   #shuffling the data
np.random.shuffle(test_data)

In [5]:
train_images = []              #separating the image and labels from the train_data list
train_labels = []
for features, labels in train_data:
    train_images.append(features)
    train_labels.append(labels)

In [7]:
test_images = []    #separating the image and labels from the test_data list
test_labels = []
for features, labels in test_data:
    test_images.append(features)
    test_labels.append(labels)

In [8]:
label_enc = LabelEncoder()           # encoding the labels 
train_labels = label_enc.fit_transform(train_labels)
test_labels = label_enc.transform(test_labels)

In [9]:
train_images = np.array(train_images)    #converting the images and labels into numpy array
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [10]:
train_images = train_images/255    # normalizing the image pixels
test_images = test_images/255

In [11]:
train_images = np.expand_dims(train_images, axis=3)      # adding a dimension on the images
test_images = np.expand_dims(test_images, axis=3)

In [12]:
print(f"Shape of the train images {train_images.shape}")
print(f"Shape of the train labels {train_labels.shape}")
print(f"Shape of the test images {test_images.shape}")
print(f"Shape of the test labels {test_labels.shape}")

Shape of the train images (2540, 150, 150, 1)
Shape of the train labels (2540,)
Shape of the test images (300, 150, 150, 1)
Shape of the test labels (300,)


In [17]:
model1 = Sequential()

model1.add(Conv2D(32, (3, 3), input_shape=(150,150,1), activation="relu"))
model1.add(MaxPool2D(2,2))
model1.add(Conv2D(64, (3, 3), activation="relu"))
model1.add(MaxPool2D(2,2))
model1.add(Conv2D(128, (3, 3), activation="relu"))
model1.add(MaxPool2D(2,2))
model1.add(Conv2D(256, (3, 3), activation="relu"))
model1.add(MaxPool2D(2,2))
model1.add(Flatten())
model1.add(Dense(256, activation="relu"))
model1.add(Dense(1, activation="sigmoid"))

In [18]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 256)      

In [19]:
model1.compile(optimizer="adam", loss="binary_crossentropy", metrics = ['accuracy'])

In [26]:
history1 = model1.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=10)

Epoch 1/10
80/80 [==============================] - 62s 769ms/step - loss: 0.5601 - accuracy: 0.7205 - val_loss: 0.5411 - val_accuracy: 0.7167
Epoch 2/10
80/80 [==============================] - 49s 618ms/step - loss: 0.5341 - accuracy: 0.7421 - val_loss: 0.5215 - val_accuracy: 0.7467
Epoch 3/10
80/80 [==============================] - 42s 527ms/step - loss: 0.5143 - accuracy: 0.7496 - val_loss: 0.4813 - val_accuracy: 0.7933
Epoch 4/10
80/80 [==============================] - 39s 490ms/step - loss: 0.4711 - accuracy: 0.7768 - val_loss: 0.5286 - val_accuracy: 0.7400
Epoch 5/10
80/80 [==============================] - 37s 457ms/step - loss: 0.4497 - accuracy: 0.7953 - val_loss: 0.5383 - val_accuracy: 0.7333
Epoch 6/10
80/80 [==============================] - 40s 499ms/step - loss: 0.4259 - accuracy: 0.8091 - val_loss: 0.4662 - val_accuracy: 0.7867
Epoch 7/10
80/80 [==============================] - 63s 790ms/step - loss: 0.3697 - accuracy: 0.8378 - val_loss: 0.4874 - val_accuracy: 0.7867

In [27]:
model1.save("Autism.hdf5")

In [ ]:
li=['Aut','Normal']
import tensorflow
import numpy as np
from keras.preprocessing import image
modelfile="Autism.hdf5"
model = tensorflow.keras.models.load_model(modelfile)
from os import listdir
test_dir='D:\\datasets\\AutismDataset\\test\\Non'
files = listdir(test_dir)
i=1
for f in files:
    filename=test_dir+"\\"+f
    new_img = image.load_img(filename, target_size=(150, 150),color_mode="grayscale")
    img = image.img_to_array(new_img)
    img = np.expand_dims(img, axis=0)
    img = img / 255
    pred=(model.predict(img) > 0.5).astype("int32")
    if pred[0][0]==1:
        print(i,'aut')
    else:
        print(i,'normal')
    i+=1

Using TensorFlow backend.


1 normal
2 normal
3 aut
4 normal
5 normal
6 aut
7 normal
8 normal
9 normal
10 aut
11 normal
12 normal
13 normal
14 aut
15 normal
16 normal
17 normal
18 normal
19 normal
20 normal
21 normal
22 normal
23 normal
24 normal
25 normal
26 normal
27 normal
28 normal
29 normal
30 normal
31 normal
32 normal
33 normal
34 normal
35 normal
36 normal
37 normal
38 normal
39 normal
40 normal
41 normal
42 normal
43 normal
44 normal
45 normal
46 normal
47 normal
48 normal
49 normal
50 normal
51 normal
52 normal
53 normal
54 normal
55 normal
56 normal
57 aut
58 aut
59 aut
60 normal
61 normal
62 normal
63 normal
64 normal
65 normal
66 aut
67 normal
68 normal
69 normal
70 aut
71 normal
72 normal
73 aut
74 normal
75 aut
76 aut
77 normal
78 normal
79 normal
80 aut
81 normal
82 normal
83 normal
84 normal
